# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1112]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [1113]:
# Read CSV from output from part 1.

city_load = 'output_data/cities.csv'

city_data = pd.read_csv(city_load)

city_data



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ribeira Grande,38.5167,-28.7000,15.22,67,40,2.57,PT,1644782521
1,Severo-Kuril'sk,50.6789,156.1250,-5.60,70,36,4.57,RU,1644782652
2,Isangel,-19.5500,169.2667,26.06,94,40,2.06,VU,1644782396
3,San Patricio,28.0170,-97.5169,18.42,22,0,3.10,US,1644782652
4,Hūn,29.1268,15.9477,13.21,38,0,4.36,LY,1644782562
...,...,...,...,...,...,...,...,...,...
551,Wufeng,27.4500,104.8333,0.55,94,31,0.47,CN,1644783455
552,Harper,4.3750,-7.7169,25.83,83,54,3.41,LR,1644783455
553,Nador,35.1740,-2.9287,12.14,76,0,1.03,MA,1644783455
554,Gimli,50.6336,-96.9907,-18.82,97,100,7.63,CA,1644783213


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [1114]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' and Humidity into  locations with float formatting.
locations = city_data[["Lat", "Lng"]].astype(float)

humidity = city_data["Humidity"].astype(float)



In [1115]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

# Create figure specs
figure_layout = {
    'width': '1000px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, center = (10,0), zoom_level=2)

# Format heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [1116]:

# Unit conversions from metric to degF and mph.
city_data['Max Temp'] = ((9/5) * city_data['Max Temp']) + 32
city_data['Wind Speed'] = (city_data['Wind Speed']) * 2.23694

# Dropping data outside temperature range of 70-80 degF.
ideal_temp = city_data.drop(city_data[city_data['Max Temp'] <= 70].index, inplace=True)
ideal_temp = city_data.drop(city_data[city_data['Max Temp'] >= 80].index, inplace=False)

#  Further filtering dataframe for wind speed less than 10 mph
ideal_wind_and_temp = ideal_temp.drop(ideal_temp[ideal_temp['Wind Speed'] >= 10].index, inplace=False)

# Another filter to include data with 0% cloudiness.
ideal_wind_and_temp_clds = ideal_wind_and_temp.drop(ideal_wind_and_temp[ideal_wind_and_temp['Cloudiness'] > 0].index, inplace=False)

# Create new column titled Hotel Name for placeholder
ideal_wind_and_temp_clds['Hotel Name'] = ""

# Changing name and creating dataframe.
hotel_df = ideal_wind_and_temp_clds

hotel_df = hotel_df.reset_index(drop=True)

hotel_df = pd.DataFrame(hotel_df)

hotel_df




,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Saint George,37.1041,-113.5841,70.358,18,0,1.990877,US,1644782585,
1,Seddon,-37.8068,144.8916,71.222,69,0,3.444888,AU,1644782798,
2,Salalah,17.0151,54.0924,77.090,25,0,4.608096,OM,1644782869,
3,Diffa,13.3154,12.6113,73.562,11,0,6.241063,NE,1644783011,
4,Half Moon Bay,37.4636,-122.4286,78.314,53,0,0.000000,US,1644783304,
5,Makkah al Mukarramah,21.4267,39.8261,75.668,38,0,2.460634,SA,1644783302,
6,Villa María,-32.4075,-63.2402,79.826,45,0,4.473880,AR,1644783377,
7,Illapel,-31.6308,-71.1653,78.494,48,0,9.753058,CL,1644783369,


In [1117]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Establishing search variables
params = {
"type": 'hotel',
"radius": 5000,
'keyword': 'lodging',
'key': g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    params['location'] = f'{lat}, {lng}'

# run a request using our params dictionary
    response = requests.get(base_url, params=params).json()

    results = response['results']
    
# Modifying dataframe with hotel name results.
    try:

        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

    except (KeyError, IndexError):

        print('Missing field/result....skipping.')

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [1118]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]






In [1119]:
# Ensuring Hotel Names were populated from API 
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Saint George,37.1041,-113.5841,70.358,18,0,1.990877,US,1644782585,Best Western Coral Hills
1,Seddon,-37.8068,144.8916,71.222,69,0,3.444888,AU,1644782798,The Hamptons Apartments - Port Melbourne Apart...
2,Salalah,17.0151,54.0924,77.090,25,0,4.608096,OM,1644782869,Salalah Gardens Hotel
3,Diffa,13.3154,12.6113,73.562,11,0,6.241063,NE,1644783011,ProDAF/Diffa
4,Half Moon Bay,37.4636,-122.4286,78.314,53,0,0.000000,US,1644783304,Half Moon Bay Lodge
5,Makkah al Mukarramah,21.4267,39.8261,75.668,38,0,2.460634,SA,1644783302,Fajar badie Hotel
6,Villa María,-32.4075,-63.2402,79.826,45,0,4.473880,AR,1644783377,Howard Johnson Hotel and Casino Villa Maria
7,Illapel,-31.6308,-71.1653,78.494,48,0,9.753058,CL,1644783369,Hotel Casablanca


In [1120]:
# Customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, center = (10,0), zoom_level=2)

# Creating clickable markers
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


fig.add_layer(heat_layer)
# fig.add_layer(markers)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…